In [2]:
# Cell 1: Import required libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Cell 2: Define stock tickers and download historical data
tickers = ["AAPL", "GOOG", "AMZN", "NFLX"]

# Download historical data
data = {}
for ticker in tickers:
    data[ticker] = yf.download(ticker, start="2014-01-01", end="2024-11-01")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [6]:
data['AAPL'].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,19.845715,19.893929,19.715000,19.754642,17.234303,234684800
2014-01-03,19.745001,19.775000,19.301071,19.320715,16.855722,392467600
2014-01-06,19.194643,19.528570,19.057142,19.426071,16.947645,412610800
2014-01-07,19.440001,19.498571,19.211430,19.287144,16.826443,317209200
2014-01-08,19.243214,19.484285,19.238930,19.409286,16.932997,258529600


In [7]:
def backtest_strategy(stock_data):
    stock_data['3M_High'] = stock_data['Adj Close'].rolling(window=63).max()  # 3-month high
    stock_data['MA20'] = stock_data['Adj Close'].rolling(window=20).mean()   # 20-day moving average

    # Create buy signal: 3-month high on the second occurrence
    stock_data['Buy_Signal'] = (
        (stock_data['Adj Close'] == stock_data['3M_High']) &
        (stock_data['Adj Close'].shift(1) == stock_data['3M_High'].shift(1))
    ).shift(1)

    # Forward fill buy signals for holding the position
    stock_data['Position'] = stock_data['Buy_Signal'].cumsum()
    stock_data['Sell_Signal'] = stock_data['Adj Close'] < stock_data['MA20']
    stock_data['Position'] = np.where(stock_data['Sell_Signal'], 0, stock_data['Position'])

    # Calculate returns
    stock_data['Strategy_Return'] = stock_data['Adj Close'].pct_change() * stock_data['Position'].shift(1)
    stock_data['Buy_and_Hold_Return'] = stock_data['Adj Close'].pct_change()

    return stock_data

In [8]:
backtest_results = {}
for ticker, stock_data in data.items():
    backtest_results[ticker] = backtest_strategy(stock_data)

In [9]:
strategy_returns = pd.concat([backtest_results[ticker]['Strategy_Return'] for ticker in tickers], axis=1).mean(axis=1)
buy_hold_returns = pd.concat([backtest_results[ticker]['Buy_and_Hold_Return'] for ticker in tickers], axis=1).mean(axis=1)

In [15]:
df = backtest_results['AAPL']
df[df['Buy_Signal'] == True]

,Open,High,Low,Close,Adj Close,Volume,3M_High,MA20,Buy_Signal,Position,Sell_Signal,Strategy_Return,Buy_and_Hold_Return
Date,,,,,,,,,,,,,
2014-04-28,20.457144,21.276787,20.448214,21.217501,18.621325,669485600,18.621325,16.795642,True,1.0,False,0.0,0.038728
2014-04-29,21.205000,21.285000,21.053928,21.154642,18.566153,337377600,18.621325,16.882763,True,2.0,False,-0.002963,-0.002963
2014-05-21,21.565357,21.667856,21.502144,21.653929,19.110493,196859600,19.110493,18.614073,True,3.0,False,0.005292,0.002646
2014-05-22,21.664286,21.780357,21.575001,21.688213,19.140751,200760000,19.140751,18.681294,True,4.0,False,0.00475,0.001583
2014-05-23,21.687500,21.954643,21.659643,21.933214,19.356972,232209600,19.356972,18.752790,True,5.0,False,0.045185,0.011296
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-09,227.929993,229.399994,226.369995,228.679993,228.164581,48076100,228.164581,213.756633,True,202.0,False,0.758744,0.003775
2024-07-10,229.300003,233.080002,229.250000,232.979996,232.454895,62627700,232.454895,215.745142,True,203.0,False,3.798326,0.018804
2024-07-11,231.389999,232.389999,225.770004,227.570007,227.057098,64710600,232.454895,216.763841,True,204.0,False,-4.713829,-0.023221


In [18]:
import pandas as pd
import numpy as np

# Create a simple dataset
simple_data = pd.DataFrame({
    'Date': pd.date_range(start='2023-01-01', periods=100),
    'Close': [100 + (x % 10) * 2 for x in range(100)]  # Cyclic price pattern
})
simple_data.set_index('Date', inplace=True)

# Define the backtesting strategy
def backtest_strategy_demo(stock_data):
    stock_data['3M_High'] = stock_data['Close'].rolling(window=63).max()  # 3-month high
    stock_data['MA20'] = stock_data['Close'].rolling(window=20).mean()   # 20-day moving average

    # Create buy signal: 3-month high on the second occurrence
    stock_data['Buy_Signal'] = (
        (stock_data['Close'] == stock_data['3M_High']) &
        (stock_data['Close'].shift(1) == stock_data['3M_High'].shift(1))
    ).shift(1)

    # Forward fill buy signals for holding the position
    stock_data['Position'] = stock_data['Buy_Signal'].cumsum()
    stock_data['Sell_Signal'] = stock_data['Close'] < stock_data['MA20']
    stock_data['Position'] = np.where(stock_data['Sell_Signal'], 0, stock_data['Position'])

    # Calculate returns
    stock_data['Strategy_Return'] = stock_data['Close'].pct_change() * stock_data['Position'].shift(1)
    stock_data['Buy_and_Hold_Return'] = stock_data['Close'].pct_change()

    return stock_data

# Apply the backtest strategy to the simple dataset
demo_results = backtest_strategy_demo(simple_data)

# Display results
demo_results.tail(20)

,Close,3M_High,MA20,Buy_Signal,Position,Sell_Signal,Strategy_Return,Buy_and_Hold_Return
Date,,,,,,,,
2023-03-22,100,118.0,109.0,False,0,True,-0.0,-0.152542
2023-03-23,102,118.0,109.0,False,0,True,0.0,0.020000
2023-03-24,104,118.0,109.0,False,0,True,0.0,0.019608
2023-03-25,106,118.0,109.0,False,0,True,0.0,0.019231
2023-03-26,108,118.0,109.0,False,0,True,0.0,0.018868
2023-03-27,110,118.0,109.0,False,0.0,False,0.0,0.018519
2023-03-28,112,118.0,109.0,False,0.0,False,0.0,0.018182
2023-03-29,114,118.0,109.0,False,0.0,False,0.0,0.017857
2023-03-30,116,118.0,109.0,False,0.0,False,0.0,0.017544
